#Cloudbakers Data Extraction Script.

This script connects to BigQuery using to query the Github Archive for the initial dataset.
The initial dataset is then restructured on a weekly basis with event counts as variables. 
Additionaly a Binary Status Variable is created to indicate a weeks Acitive/Inactive status.


In [2]:
### Initalizing libraries

import warnings
import pandas as pd
warnings.filterwarnings("ignore")
#pip install config
import json
import requests
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
#import config  #note use pip install config if this gives an error
from termcolor import colored
import threading
import time
import seaborn as sns

#### Authorize google account to use BigQuery
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

### Connect to the required BigQuery project id
project_id = 'capstone-303902'
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

### Authorize to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/My_Capstone')

### Github configuration using API keys.
Username = 'github-repo-health-uic'
Password = '817569a8eea8173f3fa928d76caada707036ad99'
github_api = "https://api.github.com"
gh_session = requests.Session()
gh_session.auth = (Username, Password)

### Reading the Test Repositories and converting to a list
Test_repos = pd.read_excel('TestRepos_300.xlsx')
id = Test_repos.iloc[0:304:]
id=id['id'].tolist()

Authenticated
Mounted at /content/drive


In [8]:
##### Creating a BigQuery Job to extract data from Github Archive for the 300 Test Repositories.

### SQL Code to get the count of each events 

sql='''
SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, repo_name, type as Eevnt_Type, count (*) Event_Count
FROM
(
SELECT type,created_at as date, repo.id as repoID, repo.name as repo_name
FROM githubarchive.year.2011
WHERE type IN ('PushEvent','CreateEvent','WatchEvent','IssueCommentEvent','PullRequestEvent','IssuesEvent','ForkEvent','CommitComentEvent') AND repo.id IN UNNEST(@ids)    
UNION ALL     
SELECT type,created_at as date, repo.id as repoID, repo.name as repo_name
FROM githubarchive.year.2012
WHERE type IN ('PushEvent','CreateEvent','WatchEvent','IssueCommentEvent','PullRequestEvent','IssuesEvent','ForkEvent','CommitComentEvent') AND repo.id IN UNNEST(@ids)    
UNION ALL 
SELECT type,created_at as date, repo.id as repoID, repo.name as repo_name
FROM githubarchive.year.2013
WHERE type IN ('PushEvent','CreateEvent','WatchEvent','IssueCommentEvent','PullRequestEvent','IssuesEvent','ForkEvent','CommitComentEvent') AND repo.id IN UNNEST(@ids)    
UNION ALL     
SELECT type,created_at as date, repo.id as repoID, repo.name as repo_name
FROM githubarchive.year.2014
WHERE type IN ('PushEvent','CreateEvent','WatchEvent','IssueCommentEvent','PullRequestEvent','IssuesEvent','ForkEvent','CommitComentEvent') AND repo.id IN UNNEST(@ids)    
UNION ALL 
SELECT type,created_at as date, repo.id as repoID, repo.name as repo_name
FROM githubarchive.year.2015
WHERE type IN ('PushEvent','CreateEvent','WatchEvent','IssueCommentEvent','PullRequestEvent','IssuesEvent','ForkEvent','CommitComentEvent') AND repo.id IN UNNEST(@ids)
UNION ALL
SELECT type,created_at as date, repo.id as repoID, repo.name as repo_name
FROM githubarchive.year.2016
WHERE type IN ('PushEvent','CreateEvent','WatchEvent','IssueCommentEvent','PullRequestEvent','IssuesEvent','ForkEvent','CommitComentEvent') AND repo.id IN UNNEST(@ids)
UNION ALL
SELECT type,created_at as date, repo.id as repoID, repo.name as repo_name
FROM githubarchive.year.2017
WHERE type IN ('PushEvent','CreateEvent','WatchEvent','IssueCommentEvent','PullRequestEvent','IssuesEvent','ForkEvent','CommitComentEvent') AND repo.id IN UNNEST(@ids)
UNION ALL
SELECT type,created_at as date, repo.id as repoID, repo.name as repo_name
FROM githubarchive.year.2018
WHERE type IN ('PushEvent','CreateEvent','WatchEvent','IssueCommentEvent','PullRequestEvent','IssuesEvent','ForkEvent','CommitComentEvent') AND repo.id IN UNNEST(@ids)
UNION ALL
SELECT type,created_at as date, repo.id as repoID, repo.name as repo_name
FROM githubarchive.year.2019
WHERE type IN ('PushEvent','CreateEvent','WatchEvent','IssueCommentEvent','PullRequestEvent','IssuesEvent','ForkEvent','CommitComentEvent') AND repo.id IN UNNEST(@ids)
)
GROUP BY 1,2,3,4
ORDER BY 1;'''

### Job configuration to the bigquery
job_configuration = bigquery.QueryJobConfig(query_parameters=[bigquery.ArrayQueryParameter("ids", "INT64", id),])

### Query using the sql and jobconfiguration to convert result to dataframe
dfx = client.query(
    sql,
    job_config=job_configuration
    ).to_dataframe()

### The resulting dataset is saved, so we one does not have to run a the query every single instance- reduces cost
dfx.to_csv('/content/drive/My Drive/My_Capstone/Queried_test_300_Data.csv', index=False)

In [11]:
### Reading the saved dataset 
Dataset=pd.read_csv('/content/drive/My Drive/My_Capstone/Queried_test_300_Data.csv')
Dataset.shape
Dataset.head()

,year_month,repoID,repo_name,Eevnt_Type,Event_Count
0,2011-02-12,206402,apache/httpclient,ForkEvent,1
1,2011-02-12,462292,christkv/node-mongodb-native,ForkEvent,1
2,2011-02-12,309009,mongoid/mongoid,PushEvent,1
3,2011-02-12,108110,mongodb/mongo,PullRequestEvent,1
4,2011-02-12,462292,christkv/node-mongodb-native,WatchEvent,3


In [13]:
#### Pivoting the dataset to create event count variables 
#### Additionally, creating a binary status variable

Restruct_Df=pd.DataFrame()
counter=1

for i in range(0,len(id)):
  print(colored(counter,color='white'))
  counter+=1

  dfPushes = Dataset.loc[(Dataset['Eevnt_Type'] == 'PushEvent') & (Dataset['repoID']==id[i])]
  dfPushes.rename(columns={'Event_Count':'number_of_pushes'}, inplace=True)
  repo_name_list = dfPushes['repo_name'].unique().tolist()

  dfForks = Dataset.loc[(Dataset['Eevnt_Type'] == 'ForkEvent') & (Dataset['repoID']==id[i])]
  dfForks.rename(columns={'Event_Count':'number_of_forks'}, inplace=True)
  
  dfPulls = Dataset.loc[(Dataset['Eevnt_Type'] == 'PullRequestEvent') & (Dataset['repoID']==id[i])]
  dfPulls.rename(columns={'Event_Count':'number_of_pulls'}, inplace=True)
  
  dfBookmarks = Dataset.loc[(Dataset['Eevnt_Type'] == 'WatchEvent') & (Dataset['repoID']==id[i])]
  dfBookmarks.rename(columns={'Event_Count':'number_of_bookmarks'}, inplace=True)

  dfIssues = Dataset.loc[(Dataset['Eevnt_Type'] == 'IssuesEvent') & (Dataset['repoID']==id[i])]
  dfIssues.rename(columns={'Event_Count':'number_of_issues'}, inplace=True)

  dfIssuecomment = Dataset.loc[(Dataset['Eevnt_Type'] == 'IssueCommentEvent') & (Dataset['repoID']==id[i])]
  dfIssuecomment.rename(columns={'Event_Count':'issue_comment_count'}, inplace=True)

  dfCreateEvent = Dataset.loc[(Dataset['Eevnt_Type'] == 'CreateEvent') & (Dataset['repoID']==id[i])]
  dfCreateEvent.rename(columns={'Event_Count':'CreateEvent_count'}, inplace=True)

  merge1 = pd.merge(dfPushes,dfForks,how='outer', on='year_month')
  merge2 = pd.merge(merge1, dfPulls, how='outer', on='year_month')
  merge3 = pd.merge(merge2, dfBookmarks, how='outer', on='year_month')
  merge4 = pd.merge(merge3, dfIssues, how='outer', on='year_month')
  merge5 = pd.merge(merge4, dfIssuecomment,how='outer',on='year_month')
  merge6 = pd.merge(merge5, dfCreateEvent, how='outer',on='year_month')
  
  merge6.drop(['repoID_x','repoID_y','Eevnt_Type_x','Eevnt_Type_y','Eevnt_Type', 'repo_name_x','repo_name_y', 'repo_name'], axis=1, inplace=True) 
  merge6['repoID']=id[i]
  
  merge6.sort_values(by='year_month',ascending=True,inplace=True)
  
  merge6['year_month']=pd.to_datetime(merge6['year_month'])
  merge6.rename({'year_month': 'datetime'}, axis=1, inplace=True)
  merge6.index=merge6['datetime']
  merge6.drop('datetime',axis=1,inplace=True)

  WeeklyTS = pd.DataFrame()
  WeeklyTS = merge6.resample('W').sum()
  WeeklyTS.reset_index(level=0, inplace=True)

  WeeklyTS.drop('repoID', axis=1, inplace=True)
  WeeklyTS.fillna(0, inplace=True)
  WeeklyTS['Total_count'] = WeeklyTS.drop('datetime',axis=1).sum(axis=1)
  
  #### Setting Threshold 10% in terms of percentage of drop
  Threshold = 0.1
  for j in range(5,len(WeeklyTS)):
    Avg = WeeklyTS['Total_count'].iloc[j-5:j,].mean()
    diff = WeeklyTS['Total_count'].iloc[j] - WeeklyTS['Total_count'].iloc[j-1]
    diff1 = WeeklyTS['Total_count'].iloc[j] - Avg
    if diff>=0:
      WeeklyTS.loc[j,'Status'] = 1
    else:
      if abs(diff1)/Avg < 0.1:
        WeeklyTS.loc[j,'Status'] = 1
      else:
        WeeklyTS.loc[j,'Status'] = 0
  

  WeeklyTS.index = WeeklyTS['datetime']
  WeeklyTS.drop('datetime', axis=1, inplace=True)
  WeeklyTS.insert(0, 'year', WeeklyTS.index.year)
  WeeklyTS.insert(1, 'month', WeeklyTS.index.month)
  WeeklyTS.insert(2, 'day', WeeklyTS.index.day)
  WeeklyTS.reset_index(level=0, inplace=True)
  WeeklyTS['repoID']=id[i]
  print(colored('Completed for repo-id: %d'%id[i], color='green'))

  Restruct_Df=Restruct_Df.append(WeeklyTS, ignore_index=True)

1
Completed for repo-id: 108110
2
Completed for repo-id: 17165658
3
Completed for repo-id: 2446718
4
Completed for repo-id: 99919302
5
Completed for repo-id: 206417
6
Completed for repo-id: 45721011
7
Completed for repo-id: 31006158
8
Completed for repo-id: 39464018
9
Completed for repo-id: 462292
10
Completed for repo-id: 156630873
11
Completed for repo-id: 118030974
12
Completed for repo-id: 21193524
13
Completed for repo-id: 164026325
14
Completed for repo-id: 151294653
15
Completed for repo-id: 212382406
16
Completed for repo-id: 102083576
17
Completed for repo-id: 108125
18
Completed for repo-id: 832680
19
Completed for repo-id: 81355383
20
Completed for repo-id: 132600378
21
Completed for repo-id: 168234055
22
Completed for repo-id: 206412
23
Completed for repo-id: 32199982
24
Completed for repo-id: 32848140
25
Completed for repo-id: 19244865
26
Completed for repo-id: 70746484
27
Completed for repo-id: 93486496
28
Completed for repo-id: 45721011
29
Completed for repo-id: 108050
3

In [ ]:
#### Restructu_df is the final restructured dataset with weekly count and the status variable
#### saving this dataset into drive
Restruct_Df.to_csv("Pivot_Dataset.csv")

In [14]:
Restruct_Df.shape #(47672, 14)
Restruct_Df.groupby('Status').count()

(47672, 14)